In [1]:
!pip install sklearn2pmml
!pip install --upgrade nyoka
!pip install keras-swa

     |████████████████████████████████| 5.6MB 3.9MB/s 
  Created wheel for sklearn2pmml: filename=sklearn2pmml-0.55.3-cp36-none-any.whl size=5639682 sha256=7e8b36fe5e7ee6344f2a2987e93bd14e053bec5f676aae6a6d980b7d9e3575aa
  Stored in directory: /root/.cache/pip/wheels/c0/fe/37/69cc827c23a819fa34d35afee0cb8ffe841f05f54c0f3c0e3d
Successfully built sklearn2pmml
     |████████████████████████████████| 337kB 4.8MB/s 
     |████████████████████████████████| 81kB 3.8MB/s 
  Created wheel for keras-swa: filename=keras_swa-0.1.5-cp36-none-any.whl size=9712 sha256=8ab0efdb0b3f6d425292cdb27a2ddb65e0baf43b0f39ec5ae4de77a4a30ba759
  Stored in directory: /root/.cache/pip/wheels/30/de/a7/f2a71861bb678729a45e8995dc2496118e875e64b89379c530
Successfully built keras-swa


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.neural_network import MLPClassifier
from sklearn.metrics.classification import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import randint as sp_randint

from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
from keras.layers import Dense, Input
from keras.models import Model
from keras import optimizers

from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def calculate_metrics(y_test, y_pred, target, model_name):
    horizons = y_test.columns.values
  
    metrics = pd.DataFrame(classification_report(y_test.squeeze(), y_pred, output_dict=True))
    index = [np.array([model_name for i in range(4)]), metrics.index]
    metrics = metrics.set_index(index)

    return metrics

In [0]:
X_train = pd.read_csv("drive/My Drive/Master/BigData/RecSysProduct/training_data/X_train.csv")
y_train = pd.read_csv("drive/My Drive/Master/BigData/RecSysProduct/training_data/y_train.csv")

X_test = pd.read_csv('drive/My Drive/Master/BigData/RecSysProduct/training_data/X_test.csv')
y_test = pd.read_csv('drive/My Drive/Master/BigData/RecSysProduct/training_data/y_test.csv')

X_train.drop(["idx", "man"], axis=1, inplace=True)
X_test.drop(["idx", "man"], axis=1, inplace=True)

y_train.drop(["idx"], axis=1, inplace=True)
y_test.drop(["idx"], axis=1, inplace=True)


In [0]:
parameter_space = {
    'hidden_layer_sizes': [(sp_randint.rvs(20,500,1),sp_randint.rvs(2,500,1),), 
                                          (sp_randint.rvs(10,500,1),),
                            (sp_randint.rvs(10,1000,1), sp_randint.rvs(10,500,1), sp_randint.rvs(10,500,1)),
                            (sp_randint.rvs(10,500,1), sp_randint.rvs(10,500,1), sp_randint.rvs(10,500,1), sp_randint.rvs(10,500,1))],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': [0.1, 0.01, 0.05, 0.001, 0.2],
    'max_iter': [200, 400, 600, 800, 1000]
}


nn_model =  MLPClassifier(early_stopping=True)

In [0]:

clf = RandomizedSearchCV(nn_model, parameter_space, n_jobs=-1, cv=2, n_iter=5)

In [0]:
%%time
search = clf.fit(X_train, y_train)

In [0]:
search.best_params_

{'activation': 'tanh',
 'alpha': 0.05,
 'hidden_layer_sizes': (42, 32, 164, 407),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_iter': 400,
 'solver': 'adam'}

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

pipeline_obj = Pipeline([
    ('scaler', MinMaxScaler()),
    ('nn',MLPClassifier(activation="tanh", alpha=0.05, hidden_layer_sizes=(63, 15), learning_rate="adaptive", learning_rate_init=0.05, max_iter=200, solver="sgd", early_stopping=True))
])
pipeline_obj.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('nn',
                 MLPClassifier(activation='tanh', alpha=0.05, batch_size='auto',
                               beta_1=0.9, beta_2=0.999, early_stopping=True,
                               epsilon=1e-08, hidden_layer_sizes=(63, 15),
                               learning_rate='adaptive',
                               learning_rate_init=0.05, max_fun=15000,
                               max_iter=200, momentum=0.9, n_iter_no_change=10,
                               nesterovs_momentum=True, power_t=0.5,
                               random_state=None, shuffle=True, solver='sgd',
                               tol=0.0001, validation_fraction=0.1,
                               verbose=False, warm_start=False))],
         verbose=False)

In [0]:
from nyoka import skl_to_pmml
skl_to_pmml(pipeline_obj,X_train.columns,'label',"nn_pmml.pmml")

In [0]:
calculate_metrics(y_test, pipeline_obj.predict(X_test), "label", "NN")

0             1  ...      macro avg   weighted avg
NN precision      0.782614      0.760302  ...       0.771458       0.771899
   recall         0.776728      0.766491  ...       0.771610       0.771812
   f1-score       0.779660      0.763384  ...       0.771522       0.771844
   support    85761.000000  79239.000000  ...  165000.000000  165000.000000

[4 rows x 5 columns]